In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from config import gcp_creds, gcs_taxi_uri

In [2]:
# must set up configurations before building the SparkSession
credentials_location = gcp_creds

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "C:\spark\jars\gcs-connector-hadoop3-latest.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [3]:
# create a SparkContext with the above configuration/new parameters
# SparkContext is an entry point to Spark functionality
#  - https://sparkbyexamples.com/spark/spark-sparkcontext/
#  - https://spark.apache.org/docs/3.2.0/api/java/org/apache/spark/SparkContext.html
sc = SparkContext(conf=conf)

In [4]:
# 
hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

In [5]:
# create the Spark session
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [6]:
# get the data from GCS
df_green = spark.read.parquet(gcs_taxi_uri + '/parquet/green/*/*')

In [7]:
# make sure we got it
df_green.count()

2304517

In [8]:
df_green.show(3)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-22 13:18:32|  2020-01-22 13:45:58|                 N|         1|         244|          41|              1|         5.22|       22.0|  0.0|    0.